In [1]:
import xlrd
import numpy as np
import pandas as pd
from xgboost.sklearn import XGBRegressor
from xgboost.sklearn import XGBClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [2]:
def re_cate(label):
    if label >37.2 and label < 37.6:
        return 1
    else:
        return 0

In [3]:
raw_df = pd.read_csv("./dataset/moment_table.csv", header = None)
raw_df.rename(columns = {0:'moment', 1:'flatness', 2:'distance'}, inplace = True)
raw_df['distance'] = raw_df.apply(lambda x : re_cate(x.distance), axis = 1)
raw_label = raw_df['distance'].as_matrix()
raw_df.drop(columns = ['distance'], axis = 1, inplace = True)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.


In [12]:
raw_label

array([0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0])

In [4]:
raw_feature = raw_df.as_matrix()
raw_feature_mean = raw_feature.mean(axis = 0)
raw_feature_std = raw_feature.std(axis = 0)
raw_feature_normalized = (raw_feature - raw_feature_mean)/raw_feature_std

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [5]:
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=7)

In [6]:
lr = LogisticRegression(solver='newton-cg', multi_class='multinomial', C=4, tol=1e-6, max_iter=20)

In [16]:
count_CV = 0
test_acc_record = []
test_pre_record = []
accuracy_list = []
precision_list = []
recall_list = []
f1_list = []

for train_index,test_index in kfold.split(raw_feature_normalized, raw_label):
    lr.fit(raw_feature_normalized[train_index], raw_label[train_index])
    pred_smile_label = lr.predict(raw_feature_normalized[test_index])
    real_label = raw_label[test_index]
    
    TP_num = 0
    FP_num = 0
    FN_num = 0
    TN_num = 0

    
    test_count_num = 0
    real_label_index = 0
    pre_label_num = 0
    
    for label in pred_smile_label:
        if label == real_label[real_label_index]:
            if label == 0:
                TN_num += 1
            if label == 1:
                TP_num += 1
            pre_label_num += 1
        else:
            if label == 0:
                FN_num += 1
            if label == 1:
                FP_num += 1
            
        real_label_index += 1
        test_count_num += 1
    
    print('#### In Cross Validation %d: ####'% count_CV)
    count_CV += 1
    print('NumofIns Precisely Classified : ',pre_label_num,'\t',
          'NumofIns : ',test_count_num,'\t',
          'Pre_Accuracy : ',pre_label_num/test_count_num,'\t',)
    
    test_pre_record.append(pre_label_num/test_count_num)
    print("TP:", TP_num)
    print("FP:", FP_num)
    print("FN:", FN_num)
    print("TN:", TN_num)
    accuracy = (TP_num + TN_num)/(TP_num + FP_num + FN_num + TN_num)
    precision = TP_num/(TP_num + FP_num)
    recall = TP_num/(TP_num + FN_num)
    f1 = (2 * precision * recall)/(precision + recall)
    print(accuracy)
    print(precision)
    print(recall)
    print(f1)
    accuracy_list.append(accuracy)
    precision_list.append(precision)
    recall_list.append(recall)
    f1_list.append(f1)
    
print('mean of NumofIns precisely classified',np.mean(test_pre_record))
print('mean of accuracy',np.mean(accuracy_list))
print('mean of precision',np.mean(precision_list))
print('mean of recall',np.mean(recall_list))
print('mean of f1',np.mean(f1_list))

#### In Cross Validation 0: ####
NumofIns Precisely Classified :  8 	 NumofIns :  9 	 Pre_Accuracy :  0.8888888888888888 	
TP: 6
FP: 1
FN: 0
TN: 2
0.8888888888888888
0.8571428571428571
1.0
0.923076923076923
#### In Cross Validation 1: ####
NumofIns Precisely Classified :  9 	 NumofIns :  9 	 Pre_Accuracy :  1.0 	
TP: 6
FP: 0
FN: 0
TN: 3
1.0
1.0
1.0
1.0
#### In Cross Validation 2: ####
NumofIns Precisely Classified :  7 	 NumofIns :  9 	 Pre_Accuracy :  0.7777777777777778 	
TP: 6
FP: 2
FN: 0
TN: 1
0.7777777777777778
0.75
1.0
0.8571428571428571
#### In Cross Validation 3: ####
NumofIns Precisely Classified :  7 	 NumofIns :  9 	 Pre_Accuracy :  0.7777777777777778 	
TP: 5
FP: 1
FN: 1
TN: 2
0.7777777777777778
0.8333333333333334
0.8333333333333334
0.8333333333333334
#### In Cross Validation 4: ####
NumofIns Precisely Classified :  7 	 NumofIns :  9 	 Pre_Accuracy :  0.7777777777777778 	
TP: 6
FP: 2
FN: 0
TN: 1
0.7777777777777778
0.75
1.0
0.8571428571428571
#### In Cross Validation 5: ####
N

In [8]:
SVM = SVC(kernel='rbf',decision_function_shape='ovo',C=20,shrinking =False,tol =1e-6)

In [17]:
count_CV = 0
test_acc_record = []
test_pre_record = []
accuracy_list = []
precision_list = []
recall_list = []
f1_list = []

for train_index,test_index in kfold.split(raw_feature_normalized, raw_label):
    SVM.fit(raw_feature_normalized[train_index], raw_label[train_index])
    pred_smile_label = SVM.predict(raw_feature_normalized[test_index])
    real_label = raw_label[test_index]
    
    TP_num = 0
    FP_num = 0
    FN_num = 0
    TN_num = 0

    
    test_count_num = 0
    real_label_index = 0
    pre_label_num = 0
    
    for label in pred_smile_label:
        if label == real_label[real_label_index]:
            if label == 0:
                TN_num += 1
            if label == 1:
                TP_num += 1
            pre_label_num += 1
        else:
            if label == 0:
                FN_num += 1
            if label == 1:
                FP_num += 1
            
        real_label_index += 1
        test_count_num += 1
    
    print('#### In Cross Validation %d: ####'% count_CV)
    count_CV += 1
    print('NumofIns Precisely Classified : ',pre_label_num,'\t',
          'NumofIns : ',test_count_num,'\t',
          'Pre_Accuracy : ',pre_label_num/test_count_num,'\t',)
    
    test_pre_record.append(pre_label_num/test_count_num)
    print("TP:", TP_num)
    print("FP:", FP_num)
    print("FN:", FN_num)
    print("TN:", TN_num)
    accuracy = (TP_num + TN_num)/(TP_num + FP_num + FN_num + TN_num)
    precision = TP_num/(TP_num + FP_num)
    recall = TP_num/(TP_num + FN_num)
    f1 = (2 * precision * recall)/(precision + recall)
    print(accuracy)
    print(precision)
    print(recall)
    print(f1)
    accuracy_list.append(accuracy)
    precision_list.append(precision)
    recall_list.append(recall)
    f1_list.append(f1)
    
print('mean of NumofIns precisely classified',np.mean(test_pre_record))
print('mean of accuracy',np.mean(accuracy_list))
print('mean of precision',np.mean(precision_list))
print('mean of recall',np.mean(recall_list))
print('mean of f1',np.mean(f1_list))

#### In Cross Validation 0: ####
NumofIns Precisely Classified :  8 	 NumofIns :  9 	 Pre_Accuracy :  0.8888888888888888 	
TP: 6
FP: 1
FN: 0
TN: 2
0.8888888888888888
0.8571428571428571
1.0
0.923076923076923
#### In Cross Validation 1: ####
NumofIns Precisely Classified :  9 	 NumofIns :  9 	 Pre_Accuracy :  1.0 	
TP: 6
FP: 0
FN: 0
TN: 3
1.0
1.0
1.0
1.0
#### In Cross Validation 2: ####
NumofIns Precisely Classified :  9 	 NumofIns :  9 	 Pre_Accuracy :  1.0 	
TP: 6
FP: 0
FN: 0
TN: 3
1.0
1.0
1.0
1.0
#### In Cross Validation 3: ####
NumofIns Precisely Classified :  9 	 NumofIns :  9 	 Pre_Accuracy :  1.0 	
TP: 6
FP: 0
FN: 0
TN: 3
1.0
1.0
1.0
1.0
#### In Cross Validation 4: ####
NumofIns Precisely Classified :  8 	 NumofIns :  9 	 Pre_Accuracy :  0.8888888888888888 	
TP: 5
FP: 0
FN: 1
TN: 3
0.8888888888888888
1.0
0.8333333333333334
0.9090909090909091
#### In Cross Validation 5: ####
NumofIns Precisely Classified :  8 	 NumofIns :  9 	 Pre_Accuracy :  0.8888888888888888 	
TP: 5
FP: 0
FN: 1


/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default 

In [10]:
XGB_C = XGBClassifier(
    #booster = 'gblinear',
    #objective='multi:softmax',
    #num_class=7,#必须要考虑到0的情况。这个数据集里面没有零
    n_estimators=200,
    max_depth=4,
    min_child_weight = 5,
    scale_pos_weight = 5,
    num_boost_round =5,
    max_delta_step=1000,
    alpha =2,
    eta=1
    #colsample_bytree=0.9
    #gamma=5,
    #process_type='update'
)

In [18]:
count_CV = 0
test_acc_record = []
test_pre_record = []
accuracy_list = []
precision_list = []
recall_list = []
f1_list = []

for train_index,test_index in kfold.split(raw_feature_normalized, raw_label):
    XGB_C.fit(raw_feature_normalized[train_index], raw_label[train_index])
    pred_smile_label = XGB_C.predict(raw_feature_normalized[test_index])
    real_label = raw_label[test_index]
    
    TP_num = 0
    FP_num = 0
    FN_num = 0
    TN_num = 0

    
    test_count_num = 0
    real_label_index = 0
    pre_label_num = 0
    
    for label in pred_smile_label:
        if label == real_label[real_label_index]:
            if label == 0:
                TN_num += 1
            if label == 1:
                TP_num += 1
            pre_label_num += 1
        else:
            if label == 0:
                FN_num += 1
            if label == 1:
                FP_num += 1
            
        real_label_index += 1
        test_count_num += 1
    
    print('#### In Cross Validation %d: ####'% count_CV)
    count_CV += 1
    print('NumofIns Precisely Classified : ',pre_label_num,'\t',
          'NumofIns : ',test_count_num,'\t',
          'Pre_Accuracy : ',pre_label_num/test_count_num,'\t',)
    
    test_pre_record.append(pre_label_num/test_count_num)
    print("TP:", TP_num)
    print("FP:", FP_num)
    print("FN:", FN_num)
    print("TN:", TN_num)
    accuracy = (TP_num + TN_num)/(TP_num + FP_num + FN_num + TN_num)
    precision = TP_num/(TP_num + FP_num)
    recall = TP_num/(TP_num + FN_num)
    f1 = (2 * precision * recall)/(precision + recall)
    print(accuracy)
    print(precision)
    print(recall)
    print(f1)
    accuracy_list.append(accuracy)
    precision_list.append(precision)
    recall_list.append(recall)
    f1_list.append(f1)
    
print('mean of NumofIns precisely classified',np.mean(test_pre_record))
print('mean of accuracy',np.mean(accuracy_list))
print('mean of precision',np.mean(precision_list))
print('mean of recall',np.mean(recall_list))
print('mean of f1',np.mean(f1_list))

#### In Cross Validation 0: ####
NumofIns Precisely Classified :  8 	 NumofIns :  9 	 Pre_Accuracy :  0.8888888888888888 	
TP: 6
FP: 1
FN: 0
TN: 2
0.8888888888888888
0.8571428571428571
1.0
0.923076923076923
#### In Cross Validation 1: ####
NumofIns Precisely Classified :  8 	 NumofIns :  9 	 Pre_Accuracy :  0.8888888888888888 	
TP: 6
FP: 1
FN: 0
TN: 2
0.8888888888888888
0.8571428571428571
1.0
0.923076923076923
#### In Cross Validation 2: ####
NumofIns Precisely Classified :  7 	 NumofIns :  9 	 Pre_Accuracy :  0.7777777777777778 	
TP: 6
FP: 2
FN: 0
TN: 1
0.7777777777777778
0.75
1.0
0.8571428571428571
#### In Cross Validation 3: ####
NumofIns Precisely Classified :  8 	 NumofIns :  9 	 Pre_Accuracy :  0.8888888888888888 	
TP: 6
FP: 1
FN: 0
TN: 2
0.8888888888888888
0.8571428571428571
1.0
0.923076923076923
#### In Cross Validation 4: ####
NumofIns Precisely Classified :  8 	 NumofIns :  9 	 Pre_Accuracy :  0.8888888888888888 	
TP: 6
FP: 1
FN: 0
TN: 2
0.8888888888888888
0.8571428571428571
